In [1]:
!nvidia-smi

Sat Dec 18 12:07:56 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install yfinance

     |████████████████████████████████| 6.4 MB 6.5 MB/s 
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [3]:
import yfinance as yf
import numpy as np
import pandas as pd
import tensorflow as tf

In [8]:
data = yf.download("MSFT" , start = "2020-01-01" , interval = '1d')

[*********************100%***********************]  1 of 1 completed


In [9]:
data.shape

(496, 6)

In [10]:
data.head(4)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-02,158.779999,160.729996,158.330002,160.619995,157.615112,22622100
2020-01-03,158.320007,159.949997,158.059998,158.619995,155.652527,21116200
2020-01-06,157.080002,159.100006,156.509995,159.029999,156.054871,20813700
2020-01-07,159.320007,159.669998,157.320007,157.580002,154.631973,21634100


#### Understanding Trends within the Data  

In [11]:
data.sort_index(inplace = True)

In [12]:
# Discard all duplicate index

data = data.loc[~data.index.duplicated(keep='first')]

In [13]:
data.tail(4)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-12-14,333.220001,334.640015,324.109985,328.339996,328.339996,44438700
2021-12-15,328.609985,335.190002,324.500000,334.649994,334.649994,35381100
2021-12-16,335.709991,336.760010,323.019989,324.899994,324.899994,35034800
2021-12-17,320.880005,324.920013,317.250000,323.799988,323.799988,47750300


In [14]:
# Check for missing values

data.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [15]:
data.describe()

,Open,High,Low,Close,Adj Close,Volume
count,496.000000,496.000000,496.000000,496.000000,496.000000,4.960000e+02
mean,232.398891,234.837924,230.012419,232.573185,230.771361,3.205809e+07
std,50.256348,50.076459,50.300166,50.256739,51.074923,1.450892e+07
min,137.009995,140.570007,132.520004,135.419998,133.249527,1.055060e+07
25%,199.820004,203.152500,196.514999,200.584999,198.353668,2.274840e+07
50%,221.869995,223.930000,217.974998,221.274994,219.084503,2.791150e+07
75%,266.032494,266.934990,264.512497,265.329994,264.339386,3.616935e+07
max,344.619995,349.670013,342.200012,343.109985,343.109985,9.701270e+07


In [19]:
import plotly.graph_objects as go

# Check the trend in Closing Values

fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index , y = data['Close'] , mode = 'lines'))
fig.update_layout(height = 500 , width = 900,
                  xaxis_title='Date' , yaxis_title='Close')
fig.show()

In [21]:
from sklearn.preprocessing import MinMaxScaler
import pickle
from tqdm.notebook import tnrange

In [22]:
# Filter only required data

data = data[['Close' , 'Volume']]
data.head(4)

,Close,Volume
Date,,
2020-01-02,160.619995,22622100
2020-01-03,158.619995,21116200
2020-01-06,159.029999,20813700
2020-01-07,157.580002,21634100


In [23]:
# Confirm the Testing Set LENGTH
test_length = data[(data.index >= '2021-11-01')]

In [27]:
def CreateFeatures_and_Targets(data, feature_length):
    X = []
    Y = []

    for i in tnrange(len(data) - feature_length):
        X.append(data.iloc[i : i + feature_length,:])
        Y.append(data["Close"].values[i+feature_length])

    X = np.array(X)
    Y = np.array(Y)

    return X , Y

In [28]:
X , Y = CreateFeatures_and_Targets(data , 32)

  0%|          | 0/464 [00:00<?, ?it/s]

In [29]:
# Check the Shape

X.shape , Y.shape

((464, 32, 2), (464,))

In [34]:
Xtrain , Xtest , Ytrain , Ytest = X[:-test_length] , X[-test_length:] , Y[:-test_length] , Y[-test_length:]

TypeError: ignored